In [1]:
import cv2
from pathlib import Path
from tqdm import tqdm

In [2]:
out_dir = 'custom_train/datasets/OurDataSet/'
clips_dir = 'inputs/media/Clips/'
annotations_dir = 'inputs/annotations/'

classes = ['car', 'motorcycle', 'truck', 'priority', 'bus']

rel_dir = 'datasets/OurDataSet/'

images_out_dir = out_dir + 'images/'
labels_out_dir = out_dir + 'labels/'


In [3]:
yam_dir = out_dir + 'OurDataset.yaml'
train_dir = rel_dir + 'images/'+'train/'
val_dir = rel_dir + 'images/' + 'val/'

nc = len(classes)

In [4]:
videos = list(Path(clips_dir).rglob('*.mp4'))
videos = [str(v) for v in videos]
# videos = [str(v) for v in videos if 'train' in str(v).lower()]


In [5]:
train_paths = []
val_paths = []

for video in videos:
    # print(video)
    extension = ''
    if 'train' in video.lower():
        extension = '_Train'
    elif 'test' in video.lower():
        extension = '_Test'

    vid_dir = video.replace('.mp4', extension)

    vid_dir = vid_dir.split('\\')[-1].split('/')[-1]
    annotations_paths = [str(v) for v in list(Path(annotations_dir+vid_dir).rglob('*.txt'))]

    cap = cv2.VideoCapture(video)
    n_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    # print(fps)
    # print(n_frames)
    for i in tqdm(range(n_frames), desc='Processing '+vid_dir):
        ret, frame = cap.read()
        if ret:
            # creaye dir if not exist
            if not Path(images_out_dir+vid_dir).exists():
                Path(images_out_dir+vid_dir).mkdir(parents=True)
            cv2.imwrite(images_out_dir + vid_dir+'/' + str(i) + '.jpg', frame)
            # cv2.imshow('frame', frame)

            with open(annotations_paths[i]) as f:
                if len(f.readlines()) == 0:
                    continue
                
                # if sub_folder does not exist create it
                if not Path(labels_out_dir + vid_dir).exists():
                    Path(labels_out_dir + vid_dir).mkdir(parents=True)

                with open(labels_out_dir + vid_dir+'/' + str(i) + '.txt', 'w') as destination:
                    for line in f:
                        destination.write(line)

        else:
            print('error')
            break

Processing Wet_Dim_Train: 100%|██████████| 1205/1205 [00:14<00:00, 82.59it/s]


In [6]:
train_paths = []
val_paths = []

full_images = out_dir + 'images/'
full_labels = out_dir + 'labels/'

all_image_dirs = os.listdir(full_images)
all_train_images = [i for i in all_image_dirs if 'train' in i.lower()]
all_val_images = [i for i in all_image_dirs if 'test' in i.lower()]

all_train_images

path = '../datasets/OurDataSet/'
im_dir ='images/'
lab_dir = 'labels/'


In [8]:
yam_file = open(yam_dir, 'w')
yam_file.write('# This file is automatically generated by dataSetter.py\n')
# yam_file.write('train: ' + train_dir + '\n')
# yam_file.write('val: ' + val_dir + '\n')

yam_file.write('path: ' + path + '\n')
yam_file.write('train: ' + '\n')
for dir in all_train_images:
    yam_file.write('  - '+im_dir+dir+'\n')
if len(all_val_images) > 0:
    yam_file.write('val: ' + '\n')
    for dir in all_val_images:
        yam_file.write('  - '+im_dir+dir+'\n')
yam_file.write('\n')


yam_file.write('# number of classes\n')
yam_file.write('nc: ' + str(nc) + '\n')
yam_file.write('\n')
yam_file.write('# class names\n')
yam_file.write('names: ')
yam_file.write('[' + '\'' + '\', \''.join(classes) + '\'' + ']')


49